In [1]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.2 MB/s eta 0:00:00


In [2]:
import openai
import re
import time

import numpy as np

from tqdm import tqdm
#from datasets import load_dataset

In [3]:
openai.api_key = "sk-"

In [4]:
from openai import OpenAI
client = OpenAI(api_key=openai.api_key)

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "Please play the role of a math teacher. Please kindly answer the questions."},
    {"role": "user", "content": "what is the Newton’s law? How did World War I end?"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content="Certainly! Newton's laws of motion are a set of three fundamental principles formulated by Sir Isaac Newton in the 17th century. They describe the relationship between a body and the forces acting on it. The three laws are:\n\n1. The first law, also known as the law of inertia, states that an object at rest will remain at rest, and an object in motion will remain in motion unless acted upon by an external force.\n\n2. The second law relates the force acting on an object to its mass and acceleration. It can be stated as F = ma, where F is the force applied, m is the mass of the object, and a is the acceleration produced.\n\n3. The third law states that for every action, there is an equal and opposite reaction.\n\nAs for how World War I ended, it officially ended on November 11, 1918, with the signing of the armistice agreement between the Allies and Germany. This agreement marked the cessation of hostilities on the Western Front and eventually led to the s

Few shot engineering

영어를 한국어로 번역하는 5-shot prompt 를 작성하고 "dog"를 번역해보세요

In [6]:
completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "영어 : apple, 한국어 : 사과"},
    {"role": "system", "content": "영어 : book, 한국어 : "},
    {"role": "system", "content": "영어 : house, 한국어 : 집"},
    {"role": "system", "content": "영어 : car, 한국어 : 자동차"},
    {"role": "user", "content": "영어 : dog, 한국어 : ?"},
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='영어 : dog, 한국어 : 개', refusal=None, role='assistant', function_call=None, tool_calls=None)


자연어를 SQL 쿼리로 바꿔주는 few-shot prompt 를 작성해보세요.

In [7]:
prompt_system = '''Convert the following natural language requests into SQL queries:
1. "salary 가 50000 을 초과하는 데이터중 모든 컬럼을 employees 에서 조회하세요.": SELECT * FROM employees WHERE salary > 50000;
2. "stock이 0인 데이터중 모든 컬럼을 products 에서 조회하세요.": SELECT * FROM products WHERE stock = 0;
3. "math_score가 90을 초과하는 데이터중 name 컬럼을 students에서 조회하세요.": SELECT name FROM students WHERE math_score > 90;
4. "order_date 가 오늘을 기준으로 지난 30일 동안의 주문데이터 중 모든 컬럼을 orders 에서 조회하세요.": SELECT * FROM orders WHERE order_date >= DATE_SUB(CURDATE(), INTERVAL 30 DAY);
5. "cumstomers를 city 별로 카운트 집계해줘. ": SELECT city, COUNT(*) FROM customers GROUP BY city;
'''

prompt_user = '''
Request: "Find the average salary of employees in the marketing department."
SQL Query:
'''

completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": prompt_system},
    {"role": "user", "content": prompt_user},
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content="```sql\nSELECT AVG(salary) FROM employees WHERE department = 'Marketing';\n```", refusal=None, role='assistant', function_call=None, tool_calls=None)


Chain of Thought

다음 문제를 푸는 Chain of thought prompt 를 작성해보세요.

# Simple - 1
Solve the following problem step-by-step: 23 + 47

Step-by-step solution:
1. Write your Prompt
2. Write your Prompt
3. Write your Prompt
4. Write your Prompt

Answer: 70



# Simple - 2
Solve the following problem step-by-step: 123 - 58

Step-by-step solution:
1. Write your Prompt
2. Write your Prompt
3. Write your Prompt
4. Write your Prompt

Answer: 65

위에서 작성한 Simple - 1, Simple - 2 를 few shot 으로 활용하고 아래 문제를 질문으로 하는 프롬프트를 작성하고 결과를 확인해보세요.

In [8]:
prompt_simple1 = '''
Solve the following problem step-by-step: 23 + 47

Step-by-step solution:

Break down the numbers by place value:

23 can be broken down into 20 (tens place) and 3 (ones place).
47 can be broken down into 40 (tens place) and 7 (ones place).
Add the tens place values together:

20 (from 23) + 40 (from 47) = 60.
Add the ones place values together:

3 (from 23) + 7 (from 47) = 10.
Combine the sums from the tens and ones place values:

60 (tens sum) + 10 (ones sum) = 70.
Answer: 70
'''

prompt_simple2 = '''
Solve the following problem step-by-step: 123 - 58

Step-by-step solution:

Break down the numbers into place values:

123 can be broken down into 100 (hundreds place), 20 (tens place), and 3 (ones place).
58 can be broken down into 50 (tens place) and 8 (ones place).
Subtract the tens place values:

20 (from 123) - 50 (from 58). To handle this, first find the difference between 20 and 50, which is -30.
Subtract the ones place values:

3 (from 123) - 8 (from 58). This results in -5.
Combine the results from the tens and ones place values:

Adjust the subtraction of the tens place by adding 100 (to account for borrowing), making it 100 - 30 = 70.
Now, subtract the ones place difference: 70 - 5 = 65.
Answer: 65
'''

prompt_user = '''
# Simple - 2
Solve the following problem step-by-step: 123 - 58

Step-by-step solution:
1. Write your Prompt
2. Write your Prompt
3. Write your Prompt
4. Write your Prompt

Answer: 65
'''


completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": prompt_simple1},
    {"role": "system", "content": prompt_simple2},
    {"role": "user", "content": prompt_user},
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content="Let's solve the problem 123 - 58 step-by-step:\n\n1. **Break down the numbers into place values:**\n   - 123 can be broken down into 100 (hundreds place), 20 (tens place), and 3 (ones place).\n   - 58 can be broken down into 50 (tens place) and 8 (ones place).\n\n2. **Subtract the ones place values:**\n   - From the ones place: 3 (from 123) - 8 (from 58). \n   - Since 3 is less than 8, we need to borrow 1 from the tens place.\n     - The tens place in 123 becomes 1 (20 - 10 = 10).\n     - Now, we have 13 (because we added 10 to the 3).\n     - 13 - 8 = 5.\n\n3. **Subtract the tens place values (after borrowing):**\n   - With the borrowing, we have 10 (the new tens place) - 5 (from 58) = 5.\n\n4. **Subtract the hundreds place values:**\n   - Since there is no hundreds place subtraction needed in the second number (58), we take 1 for the hundreds place of 123.\n\n5. **Combine the results:**\n   - The remaining values are 1 (from the hundreds place) and 5 (f